In [1]:
import time
import flappy_bird_gym
import gymnasium
env = flappy_bird_gym.make("FlappyBird-12-v0")

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

def build_model(obs, actions):
    model = Sequential()
    
    model.add(Dense(64, activation='relu', input_shape=(1, obs)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    
    model.add(Flatten())
    model.add(Dense(actions, activation='linear'))
    model.summary()
    return model

In [3]:
obs = env.observation_space.shape[0]
actions = env.action_space.n

In [4]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [5]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=0.1, value_min=.0001, value_test=.0, nb_steps=1000000)
    memory = SequentialMemory(limit=100000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                enable_dueling_network=True, dueling_type='avg',
                nb_actions=actions, nb_steps_warmup=500)
    return dqn

In [6]:
model = build_model(obs, actions)
dqn = build_agent(model, actions)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 64)             832       
                                                                 
 dense_1 (Dense)             (None, 1, 128)            8320      
                                                                 
 dense_2 (Dense)             (None, 1, 256)            33024     
                                                                 
 dense_3 (Dense)             (None, 1, 64)             16448     
                                                                 
 dense_4 (Dense)             (None, 1, 128)            8320      
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 2)                 2

In [7]:
import warnings
warnings.simplefilter("ignore")

In [8]:
from tensorflow.keras.optimizers.legacy import Adam
#Training the Neural Network
dqn.compile(Adam(learning_rate=0.0001))

In [9]:
dqn.fit(env, nb_steps=1000000, visualize=False, verbose=1)

Training for 1000000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 74s 7ms/step - reward: -12.2000
122 episodes - episode_reward: -1000.000 [-1000.000, -1000.000] - loss: 5413.114 - mean_q: -10.422 - mean_eps: 0.099 - score: 0.001

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: -10.3000
103 episodes - episode_reward: -1000.000 [-1000.000, -1000.000] - loss: 3372.506 - mean_q: -18.352 - mean_eps: 0.099 - score: 0.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: -9.9000
99 episodes - episode_reward: -1000.000 [-1000.000, -1000.000] - loss: 1485.526 - mean_q: -24.667 - mean_eps: 0.098 - score: 0.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: -9.9000
99 episodes - episode_reward: -1000.000 [-1000.000, -1000.000] - loss: 474.388 - mean_q: -32.852 - mean_eps: 0.097 - scor

In [10]:
#Saving weights of Neural Network
#dqn.save_weights("weights/flappy_bird_solution_12.h5")

In [9]:
dqn.load_weights("weights/flappy_bird_solution_12.h5")

In [10]:
results = dqn.test(env, visualize=False, nb_episodes=100, verbose=0)

In [11]:
import numpy as np
print(np.mean(results.history['nb_steps']))
print(np.sqrt(np.cov(results.history['nb_steps'])))

822.71
642.1313829975164
